In [12]:
import numpy as np
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.metrics import classification_report 
import warnings
warnings.filterwarnings('ignore')
import mlflow.tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [13]:
df_fraud = pd.read_csv('../data/Merged_Fraud_Data.csv')
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,SEO,Safari,F,30,16778864,0,Australia
1,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,SEO,Chrome,F,34,16842045,0,Thailand
2,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,Ads,IE,M,29,16843656,0,China
3,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,16938732,0,China
4,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,16971984,0,Thailand


In [14]:
df_fraud['class'].value_counts()

class
0    116878
1     12268
Name: count, dtype: int64

In [15]:
##Normalize & Scale Numerical Features
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Select numerical features for scaling
num_features = ["purchase_value",'age']

# Apply StandardScaler (mean = 0, std = 1)
scaler = StandardScaler()
df_fraud[num_features] = scaler.fit_transform(df_fraud[num_features])

# Display scaled values
print(df_fraud[num_features].head())

   purchase_value       age
0        0.549607 -0.363124
1       -1.197335  0.101168
2        0.385831 -0.479197
3        0.986342 -0.363124
4        0.767974  0.449387


In [16]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Features to One-Hot Encode (only low-cardinality ones)
low_cardinality_features = ["source", "sex", "browser"]  # Avoid "device_id"

# Features to Label Encode (high-cardinality ones)
high_cardinality_features = ["device_id", "country"]

# One-Hot Encode only low-cardinality categorical variables
df_fraud = pd.get_dummies(df_fraud, columns=low_cardinality_features, drop_first=True)

# Label Encode high-cardinality features
label_encoder = LabelEncoder()
for feature in high_cardinality_features:
    if feature in df_fraud.columns:
        df_fraud[feature] = label_encoder.fit_transform(df_fraud[feature])


In [17]:
##Splitting the data into Features and Targets
X = df_fraud.drop(columns='class',  axis=1)
Y = df_fraud['class']

In [18]:
# Ensure the column is in datetime format
X['signup_time'] = pd.to_datetime(X['signup_time'])

# Convert to Unix timestamp (seconds since epoch)
X['signup_time'] = X['signup_time'].astype('int64') // 10**9


# Ensure the column is in datetime format
X['purchase_time'] = pd.to_datetime(X['purchase_time'])


# Convert to Unix timestamp (seconds since epoch)
X['purchase_time'] = X['purchase_time'].astype('int64') // 10**9


In [19]:
X['year'] = pd.to_datetime(X['purchase_time']).dt.year
X['month'] = pd.to_datetime(X['purchase_time']).dt.month
X['day'] = pd.to_datetime(X['purchase_time']).dt.day
X['hour'] = pd.to_datetime(X['purchase_time']).dt.hour


X['year'] = pd.to_datetime(X['signup_time']).dt.year
X['month'] = pd.to_datetime(X['signup_time']).dt.month
X['day'] = pd.to_datetime(X['signup_time']).dt.day
X['hour'] = pd.to_datetime(X['signup_time']).dt.hour


In [20]:
from imblearn.over_sampling import SMOTE
X,Y = SMOTE().fit_resample(X,Y)

In [21]:
Y.value_counts()

class
0    116878
1    116878
Name: count, dtype: int64

In [22]:
print(X.columns)

Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'age', 'ip_address', 'country', 'source_Direct',
       'source_SEO', 'sex_M', 'browser_FireFox', 'browser_IE', 'browser_Opera',
       'browser_Safari', 'year', 'month', 'day', 'hour'],
      dtype='object')


In [23]:
##Split the data into Traning data and Testing Data

#X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)


In [24]:
print (X_train)

        user_id  signup_time  purchase_time  purchase_value  device_id  \
102270   399273   1429493192     1434826571        2.733284      36597   
132404   357781   1420161439     1420161440       -0.214680      55858   
178920   308412   1420277194     1420277195        0.877158      25367   
509      285346   1434390032     1441721181        0.331239     115966   
10998    152842   1431101933     1441383337        2.405732      16413   
...         ...          ...            ...             ...        ...   
34253     29555   1421098277     1421098278       -0.815192      72468   
223744   266367   1420486720     1420486721       -0.487640      57172   
27014    359925   1431070420     1438294864        0.440423      26134   
79011     72788   1427780104     1432592675        0.276647     114723   
47333    294668   1428216672     1430313801        0.331239      44780   

             age  ip_address  country  source_Direct  source_SEO  sex_M  \
102270 -1.523855  2993528034      17

In [25]:
# define the model parameters 
params = {
    'solver': 'lbfgs',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state': 8888,
}

#Train the model 
lr = LogisticRegression(**params)
lr.fit(X_train, Y_train)

#predict on the test set 
Y_pred = lr.predict(X_test)

report = classification_report(Y_test,Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.68      0.65      0.66     23376
           1       0.66      0.69      0.68     23376

    accuracy                           0.67     46752
   macro avg       0.67      0.67      0.67     46752
weighted avg       0.67      0.67      0.67     46752



In [26]:
report_dict = classification_report(Y_test, Y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.6792063207038965,
  'recall': 0.6472450376454483,
  'f1-score': 0.6628406203452204,
  'support': 23376.0},
 '1': {'precision': 0.6630985455139728,
  'recall': 0.6943018480492813,
  'f1-score': 0.6783415531221265,
  'support': 23376.0},
 'accuracy': 0.6707734428473648,
 'macro avg': {'precision': 0.6711524331089347,
  'recall': 0.6707734428473648,
  'f1-score': 0.6705910867336735,
  'support': 46752.0},
 'weighted avg': {'precision': 0.6711524331089347,
  'recall': 0.6707734428473648,
  'f1-score': 0.6705910867336734,
  'support': 46752.0}}

Experiment 1: Train Logistic Regression Classifier

In [27]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train,Y_train)
Y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(Y_test,Y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.68      0.65      0.66     23376
           1       0.66      0.69      0.68     23376

    accuracy                           0.67     46752
   macro avg       0.67      0.67      0.67     46752
weighted avg       0.67      0.67      0.67     46752



Experiment 2: Train Random Forest Classifier

In [28]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train,Y_train)
Y_pred_rf = rf_clf.predict(X_test)
print(classification_report(Y_test,Y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.68      0.65      0.66     23376
           1       0.66      0.69      0.68     23376

    accuracy                           0.67     46752
   macro avg       0.67      0.67      0.67     46752
weighted avg       0.67      0.67      0.67     46752



Experiment 3: Train XGBoost

In [29]:
xgb_clf = XGBClassifier(use_lable_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train,Y_train)
Y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(Y_test,Y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.68      0.65      0.66     23376
           1       0.66      0.69      0.68     23376

    accuracy                           0.67     46752
   macro avg       0.67      0.67      0.67     46752
weighted avg       0.67      0.67      0.67     46752



Experiment 4: Decision Tree Classifier

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

dt_clf = DecisionTreeClassifier(max_depth=5)  # You can tune hyperparameters
dt_clf.fit(X_train, Y_train)
Y_pred_dt = dt_clf.predict(X_test)

print(classification_report(Y_test, Y_pred_dt))

              precision    recall  f1-score   support

           0       0.76      0.92      0.83     23376
           1       0.90      0.71      0.79     23376

    accuracy                           0.82     46752
   macro avg       0.83      0.82      0.81     46752
weighted avg       0.83      0.82      0.81     46752



In [32]:
dtc = DecisionTreeClassifier()
dtc.fit(X , Y)

DecisionTreeClassifier()

In [33]:
import joblib

In [34]:
joblib.dump(dtc, 'fraud_data_model.pkl')

['fraud_data_model.pkl']

In [35]:
model = joblib.load('fraud_data_model.pkl')

Track ML models Experiments Using MLFlow

In [26]:
models = [
    (
        'Logistic Regression',
        LogisticRegression(C=1, solver='liblinear'),
        (X_train,Y_train),
        (X_test,Y_test)

    ),
    (
        'Random Forest',
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train,Y_train),
        (X_test,Y_test)
    ),
    (
         'XGBClassifier',
          XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
          (X_train,Y_train),
          (X_test,Y_test)
    ),
    (
        'Decision Tree',
         DecisionTreeClassifier(max_depth=5),
        (X_train,Y_train),
        (X_test,Y_test)
    )
]

In [27]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    Y_train = train_set[1]
    X_test = test_set[0]
    Y_test = test_set[1]

    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    report = classification_report(Y_test, Y_pred,output_dict=True)
    reports.append(report)

In [29]:
mlflow.set_experiment('model tranning for fraud_data')
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]

    with mlflow.start_run(run_name = model_name):
        mlflow.log_param('model_name', model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])

        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')    

2025/02/16 12:50:47 INFO mlflow.tracking.fluent: Experiment with name 'model tranning for fraud_data' does not exist. Creating a new experiment.
2025/02/16 12:51:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/1518b25792294a828563c90c9825081e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606


2025/02/16 12:51:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/ddd42c9e63914393b2906b644fab5a36
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606


2025/02/16 12:51:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/8b6e960e80a14b6c8bb85379aaa42f06
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606


2025/02/16 12:52:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Decision Tree at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/fcbd5cafdd2c49e5a11d96aec7b30729
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606


Train  Deep learning Models

Experiment 1:Multi-Layer Perceptron (MLP) Classifier

In [31]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
mlp_clf.fit(X_train, Y_train)
Y_pred_mlp = mlp_clf.predict(X_test)

print(classification_report(Y_test, Y_pred_mlp))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     23376
           1       0.50      1.00      0.67     23376

    accuracy                           0.50     46752
   macro avg       0.25      0.50      0.33     46752
weighted avg       0.25      0.50      0.33     46752



Experiment 2 : Convolutional Neural Network (CNN)

In [35]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np

# Convert DataFrame to NumPy array
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reshape input for CNN
X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)

X_train_cnn = X_train_cnn.astype(np.float32)
X_test_cnn = X_test_cnn.astype(np.float32)


# Check if it's binary or multi-class
num_classes = len(np.unique(Y_train))

if num_classes == 2:
    # Binary classification (0/1)
    Y_train_cnn = np.array(Y_train)
    Y_test_cnn = np.array(Y_test)
    loss_function = "binary_crossentropy"
    final_activation = "sigmoid"
    output_units = 1
else:
    # Multi-class classification
    Y_train_cnn = to_categorical(Y_train, num_classes)
    Y_test_cnn = to_categorical(Y_test, num_classes)
    loss_function = "categorical_crossentropy"
    final_activation = "softmax"
    output_units = num_classes

# Define CNN model
cnn_model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dense(output_units, activation=final_activation)  # Adjust for binary/multi-class
])

cnn_model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train_cnn, Y_train_cnn, epochs=10, batch_size=32, verbose=1)

# Predictions
Y_pred_cnn = cnn_model.predict(X_test_cnn)
Y_pred_cnn_classes = (Y_pred_cnn > 0.5).astype(int) if num_classes == 2 else Y_pred_cnn.argmax(axis=1)

print(classification_report(Y_test, Y_pred_cnn_classes))



Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 29s 8ms/step - accuracy: 0.8289 - loss: 1866403.0000
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 28s 9ms/step - accuracy: 0.8253 - loss: 456893.4375
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 34s 10ms/step - accuracy: 0.8303 - loss: 219493.8906
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.8249 - loss: 114394.9766
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.8478 - loss: 33536.3516
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 36s 11ms/step - accuracy: 0.8979 - loss: 10.8453
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9051 - loss: 0.3283
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 34s 10ms/step - accuracy: 0.9058 - loss: 0.3121
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.9044 - loss: 0.3154
Epoch 10/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 32s 10ms/step - accuracy: 0.9053 - loss: 0.3132
808/808 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step
              precision    recall  

Experiment 7:Recurrent Neural Network (RNN)

In [42]:
from tensorflow.keras.layers import SimpleRNN

# # Convert DataFrame to NumPy array

X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

Y_train = np.array(Y_train, dtype=np.int32)
Y_test = np.array(Y_test, dtype=np.int32)


rnn_model = models.Sequential([
    SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.Dense(50, activation='relu'),
    layers.Dense(output_units, activation=final_activation)  # Adjusted for binary/multi-class
])

rnn_model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

rnn_model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1)

Y_pred_rnn = rnn_model.predict(X_test)
Y_pred_rnn_classes = (Y_pred_rnn > 0.5).astype(int) if num_classes == 2 else Y_pred_rnn.argmax(axis=1)

print(classification_report(Y_test, Y_pred_rnn_classes))



Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 32s 8ms/step - accuracy: 0.8302 - loss: 32598.8457
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 36s 11ms/step - accuracy: 0.8362 - loss: 73.9595
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 36s 11ms/step - accuracy: 0.8160 - loss: 30.7390
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 36s 11ms/step - accuracy: 0.8364 - loss: 2.0003
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.8851 - loss: 5.1223
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.8389 - loss: 2.8985
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.8652 - loss: 0.7916
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9032 - loss: 0.3504
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9056 - loss: 0.3198
Epoch 10/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9031 - loss: 0.5513
808/808 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
              precision    recall  f1-score   support

Experiment 8:Long Short-Term Memory (LSTM)

In [43]:
from tensorflow.keras.layers import LSTM

# # Convert DataFrame to NumPy array

X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

Y_train = np.array(Y_train, dtype=np.int32)
Y_test = np.array(Y_test, dtype=np.int32)

lstm_model = models.Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.Dense(50, activation='relu'),
    layers.Dense(output_units, activation=final_activation)  # Adjusted for binary/multi-class
])

lstm_model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

lstm_model.fit(X_train_cnn, Y_train_cnn, epochs=10, batch_size=32, verbose=1)

Y_pred_lstm = lstm_model.predict(X_test_cnn)
Y_pred_lstm_classes = (Y_pred_lstm > 0.5).astype(int) if num_classes == 2 else Y_pred_lstm.argmax(axis=1)

print(classification_report(Y_test, Y_pred_lstm_classes))


Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 57s 16ms/step - accuracy: 0.8010 - loss: 432141.8438
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 59s 18ms/step - accuracy: 0.8280 - loss: 57401.3125
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 54s 17ms/step - accuracy: 0.8267 - loss: 57181.1602
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - accuracy: 0.8267 - loss: 41752.3047
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.8520 - loss: 2774.8015
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9053 - loss: 0.3135
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9050 - loss: 0.3142
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 52s 16ms/step - accuracy: 0.9031 - loss: 0.3187
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 53s 16ms/step - accuracy: 0.9024 - loss: 9.1530
Epoch 10/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 54s 17ms/step - accuracy: 0.9046 - loss: 0.3232
808/808 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
              precision    recall  f1-

Track Deep learning models Experiments Using MLFlow

In [44]:
deep_learning_models = {
    "CNN": models.Sequential([
        layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dense(output_units, activation=final_activation)
    ]),
    "RNN": models.Sequential([
        layers.SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.Dense(50, activation='relu'),
        layers.Dense(output_units, activation=final_activation)
    ]),
    "LSTM": models.Sequential([
        layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.Dense(50, activation='relu'),
        layers.Dense(output_units, activation=final_activation)
    ])
}


In [45]:
mlflow.set_experiment('model tranning for fraud_data')
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [46]:
for model_name, model in deep_learning_models.items():
    with mlflow.start_run(run_name=model_name):
        # Compile and Train
        model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])
        model.fit(X_train_cnn, Y_train_cnn, epochs=10, batch_size=32, verbose=1)

        # Make predictions
        Y_pred = model.predict(X_test_cnn)
        Y_pred_classes = (Y_pred > 0.5).astype(int) if num_classes == 2 else Y_pred.argmax(axis=1)

        # Compute classification metrics
        report = classification_report(Y_test, Y_pred_classes, output_dict=True)

        # Log metrics
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_0': report['0']['recall'],
            'recall_class_1': report['1']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        # Log deep learning model
        mlflow.tensorflow.log_model(model, model_name)

        print(f"Logged {model_name} successfully in MLflow")



Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.8305 - loss: 2024370.8750
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 39s 12ms/step - accuracy: 0.8273 - loss: 682612.3750
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 43s 13ms/step - accuracy: 0.8284 - loss: 303261.2812
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.8286 - loss: 103756.9219
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 34s 10ms/step - accuracy: 0.8279 - loss: 38138.3555
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/step - accuracy: 0.9026 - loss: 131.0138
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9042 - loss: 0.3159
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9064 - loss: 0.3107
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 32s 10ms/step - accuracy: 0.9044 - loss: 0.3153
Epoch 10/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.9038 - loss: 0.3166
808/808 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step


2025/02/16 13:35:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/16 13:36:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged CNN successfully in MLflow
🏃 View run CNN at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/e67592a1c7884b2faf5fd08d8aeecfe8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606
Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 37s 9ms/step - accuracy: 0.8188 - loss: 12467.9385
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 32s 10ms/step - accuracy: 0.8257 - loss: 198.6428
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 31s 10ms/step - accuracy: 0.8310 - loss: 24.4148
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 34s 11ms/step - accuracy: 0.8458 - loss: 1.6816
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 37s 11ms/step - accuracy: 0.8943 - loss: 0.9636
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - accuracy: 0.8950 - loss: 0.4410
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - accuracy: 0.8999 - loss: 0.5920
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 36s 11ms/step - accuracy: 0.9040 - loss: 0.3399
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 35s 11ms/

2025/02/16 13:42:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/16 13:43:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged RNN successfully in MLflow
🏃 View run RNN at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/7ca7d1eeddec4daaab4acec65ae98c27
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606
Epoch 1/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - accuracy: 0.7937 - loss: 391872.7500
Epoch 2/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - accuracy: 0.8307 - loss: 24.4156
Epoch 3/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.8593 - loss: 13515.5186
Epoch 4/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.8311 - loss: 907.5781
Epoch 5/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - accuracy: 0.8525 - loss: 148.7984
Epoch 6/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.8813 - loss: 851.8013
Epoch 7/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - accuracy: 0.9023 - loss: 0.6637
Epoch 8/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.9047 - loss: 0.3148
Epoch 9/10
3229/3229 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms

2025/02/16 13:47:21 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/16 13:47:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged LSTM successfully in MLflow
🏃 View run LSTM at: http://127.0.0.1:5000/#/experiments/200134848443818606/runs/2ba9f4ce4379409f9fcb9686a2e6e11c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/200134848443818606
